In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import  MinMaxScaler
from sklearn.metrics import mean_squared_error
import os

In [ ]:
IS_LOCAL = False
if(IS_LOCAL):
    PATH="../input/LANL/"
else:
    PATH="../input/"
os.listdir(PATH)

In [ ]:
%%time
train_df = pd.read_csv(os.path.join(PATH,'train.csv'), dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
train_df.head()

In [ ]:
train_sample = train_df["acoustic_data"].values[:1000000]
train_sample_tf = train_df["time_to_failure"].values[:1000000]
plt.plot(train_sample)
plt.plot(train_sample_tf)
plt.show()

In [ ]:
dataset = train_df.iloc[:3000000,0].values
plt.figure(figsize=(20,10))
plt.plot(dataset)
plt.xlabel("time")
plt.ylabel("Magnitude(Ml)")
plt.title("earthquake data")
plt.show()

In [ ]:
dataset = dataset.reshape(-1,1)
dataset = dataset.astype("float32")
dataset.shape

In [ ]:
scaler = MinMaxScaler(feature_range = (0,1))
dataset= scaler.fit_transform(dataset)

In [ ]:
train_size = int(len(dataset)*0.8)
test_size =len(dataset)-train_size
train = dataset[0:train_size,:]
test = dataset[train_size:len(dataset),:]
print("train size:{},test size {}".format(len(train),len(test)))

In [ ]:
time_stemp = 50
dataX = []
dataY =[]
for i in range(len(train)-time_stemp -1):
    a = train[i:(i+time_stemp),0]
    dataX.append(a)
    dataY.append(train[i + time_stemp,0])
trainX =np.array(dataX)
trainY = np.array(dataY)

In [ ]:
dataX = []
dataY = []
for i in range(len(test)-time_stemp-1):
    a = test[i:(i+time_stemp),0]
    dataX.append(a)
    dataY.append(test[i+time_stemp,0])
testX = np.array(dataX)
testY = np.array(dataY)

In [ ]:
trainX = np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX = np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(50,input_shape=(1,time_stemp)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error',optimizer='adam')
history = model.fit(trainX,trainY,epochs = 20,batch_size = 100)

In [ ]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

trainScore = math.sqrt(mean_squared_error(trainY[0],trainPredict[:,0]))
print('Train Score : %2f RMSE '%(trainScore))
testScore = math.sqrt(mean_squared_error(testY[0],testPredict[:,0]))
print('Test Score : %2f RMSE '%(testScore))

In [ ]:
fig = plt.figure(figsize = (15,8),)
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:,:] = np.nan
trainPredictPlot[time_stemp:len(trainPredict)+time_stemp,:] = trainPredict

testPredictPlot = np.empty_like(dataset)
testPredictPlot[:,:] = np.nan
testPredictPlot[len(trainPredict)+(time_stemp*2)+2:len(dataset)] = testPredict
plt.title("Depremler(Ardışık) SL = 50 ",fontsize = 25)
plt.xlabel("Time",fontsize = 15)
plt.ylabel("Magnitude",fontsize = 15)
plt.plot(scaler.inverse_transform(dataset),color="b",label="Bütün data")
plt.plot(trainPredictPlot,color = "g",label="Deneme Datası")
plt.plot(testPredictPlot,color = "r",label="Test Datası")
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(history.history['loss'],color = 'b',label = "validation loss")
plt.title ("Test Loss SL=50")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()